<a href="https://colab.research.google.com/github/sandy0216/debate/blob/main/%E6%8E%92%E8%A3%81%E5%88%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
import csv
from ortools.graph import pywrapgraph
import numpy as np

In [ ]:
filename = '/content/drive/MyDrive/63屆寒菁/coding_test/schedule.csv'
data = []
with open(filename, newline='') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
    data.append(row)

In [ ]:
class battle():
  def __init__(self,time,battle,school):
    self.time = time
    self.battle = battle
    self.enemy = school
    self.schs = [battle.split('x')[0],battle.split('x')[1],school]
    self.judges = []

In [ ]:
schedule = []
for i in range(int(len(data)/3)):
  sch = [] # 本時段有哪些場次
  time = data[i*3]
  bat = data[i*3+1]
  school = data[i*3+2]
  for j,b in enumerate(bat):
    if(len(b)!=0):
      temp = battle(time,bat[j],school[j])
      sch.append(temp)
  schedule.append(sch)

In [ ]:
filename = '/content/drive/MyDrive/63屆寒菁/coding_test/judges.csv'
judges = []
time  = []
avoid  = []
with open(filename, newline='') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
    judges.append(row[0])
    time.append(row[1:1+len(schedule)])  #第2-5行是時間
    avoid.append(row[1+len(schedule):])
for i in range(len(judges)):
  print(i,judges[i])

0 4莊文琪
1 4鄒偉達
2 4周泰寧
3 4蔡東霖
4 4莊棨閎
5 4李曼瑄
6 4顏廷宇
7 3楊子賢
8 3李守治
9 3許人友
10 3劉祖丞
11 3黃家綸
12 3蕭家靖
13 3郭晏瑜
14 3廖哲霈
15 2秦偉翔
16 2李映璉
17 2施敦仁
18 2張雅如
19 2王詣昕
20 0江杏霙
21 0李承哲
22 1華怡茹
23 1魏子捷
24 4施玟辰
25 4游甯筑
26 3姚博翔
27 3楊榮軒
28 2劉岳軒
29 2黃柏翔
30 4陳軍宇
31 2林嘉恩
32 2陳可惠


In [ ]:
# 每個時段有哪些裁判可用
candidates = []
for i in range(4):
  candidate = []
  for j in range(len(judges)):
    if(time[j][i]=='1'):
      candidate.append(j)
  #print(candidate)
  np.random.shuffle(candidate) #亂數一下，排出來才不會太頻繁的同字頭裁同一場
  print(len(candidate),candidate) #存進去的是judges這個array對應的編號
  candidates.append(candidate)
  if(len(candidate)<3*len(schedule[i])):
    print('裁判不夠QQQQ')

21 [10, 0, 31, 11, 26, 8, 2, 23, 4, 28, 12, 27, 3, 21, 25, 16, 6, 5, 29, 14, 30]
21 [25, 11, 12, 10, 16, 26, 5, 6, 8, 23, 22, 24, 1, 30, 4, 28, 21, 29, 3, 27, 14]
15 [17, 23, 16, 9, 32, 14, 11, 22, 20, 10, 15, 21, 13, 27, 8]
15 [15, 22, 10, 32, 8, 23, 14, 16, 13, 17, 21, 27, 9, 20, 11]


In [ ]:
def get_judge(run,output_cand=False):
  # 設定本時段可用裁判、對戰
  candidate = candidates[run]
  battles = schedule[run]
  # 解flow的設定
  n_battle = len(battles)
  n_judge  = len(candidate)
  vertex = np.arange(0,n_battle+n_judge+2,1)
  st, end = int(vertex[0]), int(vertex[-1])
  battle_arr, judge_arr = vertex[1:n_battle+1], vertex[n_battle+1:-1]
  start_nodes = []
  end_nodes = []
  capacities = []
  def add_edge(st,end,cap):
    start_nodes.append(int(st))
    end_nodes.append(int(end))
    capacities.append(int(cap))
  for i in range(n_battle):
    add_edge(st,battle_arr[i],3) # 每場比賽有三個裁判
  for i in range(n_judge):
    add_edge(judge_arr[i],end,1) # 每個裁判只能裁一場
  for i,bat in enumerate(battles):
    cyc_sch = bat.schs
    if(output_cand): print(bat.battle)
    n = 0
    for j,p in enumerate(candidate):
      flag = 0
      for sch in cyc_sch:
        if(sch in avoid[p]): flag = 1
      if(flag==0):
        if(output_cand): print(judges[p],end='\t')
        add_edge(battle_arr[i],judge_arr[j],1)
        n += 1
    if(output_cand): print(n)

  # 開始解flow
  max_flow = pywrapgraph.SimpleMaxFlow()
  # Add each arc.
  for i in range(0, len(start_nodes)):
    max_flow.AddArcWithCapacity(start_nodes[i], end_nodes[i], capacities[i])
  # Find the maximum flow between node 0 and node 4.
  if max_flow.Solve(st, end) == max_flow.OPTIMAL:
    print('[Info] Get the optimal flow!')
  if(max_flow.OptimalFlow()!=3*n_battle):
    print('[Error] Fail to solve the flow network')

  # 設定本時段存裁判編號的array
  our_judge = np.empty([n_battle,3])
  our_judge_n = np.zeros(n_battle)
  for bat in schedule[run]:
    bat.judges = []

  drop = []
  for i in range(max_flow.NumArcs()):
    # print(i,max_flow.Flow(i))
    if(max_flow.Flow(i)==1 and max_flow.Head(i)!=end):
      #print(max_flow.Tail(i)-battle_arr[0],max_flow.Head(i)-judge_arr[0])
      court = max_flow.Tail(i)-battle_arr[0]
      judge = max_flow.Head(i)-judge_arr[0]
      schedule[run][court].judges.append(judges[candidate[judge]])
    if(max_flow.Flow(i)==0 and max_flow.Head(i)==end):
      #print(max_flow.Tail(i)-judge_arr[0],max_flow.Head(i))
      drop.append(judges[candidate[max_flow.Tail(i)-judge_arr[0]]])
  return drop 

In [ ]:
backup = []
for i in range(len(schedule)):
  drop = get_judge(i)
  backup.append(drop)
print(backup)

[Info] Get the optimal flow!
[Info] Get the optimal flow!
[Info] Get the optimal flow!
[Info] Get the optimal flow!
[[], [], [], []]


In [ ]:
f = open('/content/drive/MyDrive/63屆寒菁/coding_test/output/schedule_with_judge.csv','w')
for i,sch in enumerate(schedule):
  print(sch[0].time[0],end=', ,',file=f)
  if(len(backup[i])!=0):
    for d in backup[i]:
      print(d,end=',',file=f)
  print('',file=f)
  for battle in sch:
    print(f'{battle.battle},,{battle.enemy}',end=',',file=f)
  print('',file=f)
  for battle in sch:
    for j in battle.judges:
      print(j,end=',',file=f)
  print('',file=f)
f.close()

In [ ]:
f = open('/content/drive/MyDrive/63屆寒菁/coding_test/output/schedule_with_judge_transpose.csv','w')
for i,sch in enumerate(schedule):
  print(sch[0].time[0],end=', ,',file=f)
  if(len(backup[i])!=0):
    for d in backup[i]:
      print(d,end=',',file=f)
  print('',file=f)
  for battle in sch:
    print(f'{battle.battle},,{battle.enemy}',end=',',file=f)
    for j in battle.judges:
      print(j,end=',',file=f)
    print('',file=f)  
f.close()